# PERSIPAN PUSTAKA

Pustaka dipakai:
1. selenium
2. beautifulsoup4
3. paddleOCR
4. cv2
5. pandas
6. geopandas
7. shapely
8. matplotlib

In [1]:
# Library for Scrape
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs

# Visualization
from matplotlib import pyplot as plt
import cv2

# Text Recognition
from paddleocr import PaddleOCR, draw_ocr

# Polygon Manipulation
from shapely.geometry import Polygon, box

# Dataframe Management
import geopandas as gpd
import pandas as pd

# Runtime Monitoring
import time

Alur:
1. Import file SHP
2. Ekstrak polygon dari kelurahan yang ditentukan
3. Buat grid untuk mempermudah menentukan lokasi screenshot
    * Lakukan screenshot untuk dua level zoom mengetahui resolusi yang tersedia
    * ukuran grid ditentukan berdasarkan resolusi yang diperoleh
4. Buat screenshot layar berdasarkan grid yang terbentuk
5. Dapatkan nama tempat dengan ocr, sekaligus koordinat pada layar
6. Hapus teks yang merupakan menu dari Google Maps
7. Hapus teks yang berupa nama jalan
8. Klik setiap nama tempat yang didapat
9. Dapatkan koordinat POI melalui link
10. Dapatkan detail dari tempat menggunakan bs4
11. Ulangi tahapan 4 - 10 untuk setiap grid yan didapat

# CONFIG PARAMETER

In [2]:
#path shp file
path_shp = "SHPFile Kota Sukabumi/3272.shp"

#pilih kelurahan
kecamatan = "Cikole"
kelurahan = "Cikole"

#folder output
path_output = "output/"

# CONFIG SELENIUM DRIVER AND BROWSER

In [3]:
#windows option
chrome_options = Options()
chrome_options.add_argument("--start-maximized") # windows size
chrome_options.add_experimental_option("detach",True)

#zoom gmaps
zoom1 = 20
zoom2 = 20.5
zoom3 = 21

#test coordinat
lat_test = -6.7135171
long_test = 107.6786505
#x=long; y=lat

# CALCULATE GRID SIZE

In [4]:
def calculate_grid_size(x, y, zoom_level):
    x_size = (x/256)*(360/2**zoom_level)
    y_size = (y/256)*(360/2**zoom_level)
    return x_size, y_size

# MAKE A GRID FROM MAPS

In [5]:
# Fungsi untuk membagi poligon menjadi grid persegi panjang
def create_grid(polygon:gpd.geoseries.GeoSeries, x_size:int, y_size:int):
    minx, miny, maxx, maxy = float(polygon.bounds["minx"]), float(polygon.bounds["miny"]), float(polygon.bounds["maxx"]), float(polygon.bounds["maxy"])  # Mendapatkan batas-batas poligon
    
    # Menentukan jumlah grid yang dibutuhkan dalam arah x dan y
    x_steps = int((maxx - minx) / x_size)
    y_steps = int((maxy - miny) / y_size)
    
    grid = []
    
    # Membuat persegi panjang di dalam area yang tercover poligon
    for i in range(x_steps):
        for j in range(y_steps):
            # Menentukan koordinat persegi panjang
            x0 = minx + i * x_size * 0.995
            y0 = miny + j * y_size * 0.995
            x1 = x0 + x_size
            y1 = y0 + y_size
            
            # Membuat persegi panjang dengan koordinat tersebut
            rect = box(x0, y0, x1, y1)
            
            # Memeriksa apakah persegi panjang tumpang tindih dengan poligon
            if rect.intersects(polygon.values):
                grid.append(rect)
    
    return gpd.GeoDataFrame(geometry=grid, crs="EPSG:4326")

# GET CENTROID FOR EACH GRID

In [6]:
def get_centroid(grid_list:gpd):
    grid_list["centroid"] =  grid_list.geometry.centroid
    return grid_list

# GET GMAPS LINK

In [7]:
def get_gmaps_link(lat, long, zoom_level):
    return f"https://www.google.com/maps/@{lat},{long},{zoom_level}z?"

# SCREENSHOOT

In [8]:
def make_screenshot(driver:webdriver.Chrome, link:str, save_to:str):
    driver.get(link)
    driver.save_screenshot(save_to)

    return save_to

# GET IMAGE RESOLUTION

In [9]:
def get_image_resolution(image_path:str):
    img = cv2.imread(image_path)
    y,x,z = img.shape 
    return x, y #x=lat; y=long

# OCR

In [ ]:
def detect_text(image_path, bin=False, inv=False):
    # Path
    font_path = "C:/Windows/Fonts/arial.ttf"

    # Initialize OCR
    ocr = PaddleOCR(use_angle_cls= True, lang = "en", ocr_version='PP-OCRv4', vis_font_path=font_path)

    # Perform OCR
    result = ocr.ocr(image_path, cls=True, bin=bin, inv=inv)

    # Visualization
    # Get the bounding box and corresponding text
    boxes = [line[0] for line in result[0]]
    txts = [line[1][0] for line in result[0]]
    scores = [line[1][1] for line in result[0]]

    boxes_centroid = [Polygon(line).centroid for line in boxes]
    source = image_path

    return pd.DataFrame({'text': txts, 'scores': scores, 'boxes': boxes, 'centroid':boxes_centroid, 'source':source})

# SAVE IMAGE WITH BOXES

In [11]:
def save_image_with_boxes(image_path, boxes, save_to:str):
    # Load the image
    image = cv2.imread(image_path)

    image_with_boxes = draw_ocr(image, boxes)

    # Convert the image from BGR to RGB (OpenCV loads images in BGR)
    image_with_boxes = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)

    plt.imshow(image_with_boxes)
    plt.savefig(save_to)

# CLEANING TEXT

In [ ]:
def cleaning_text(df_text):
    # Hapus Menu Google

    # Hapus nama jalan

    return df_text

# COLLECT PLACE

# GET PLACE'S DETAIL

In [13]:
tag = "!4m6!3m5!1s0x2e68498154b496b1:0x8ce79800c09324a2!8m2!3d-6.9199873!4d106.9417309!16s%2Fg%2F11s7w8jwvg?entry=ttu&g_ep=EgoyMDI0MTAyOS4wIKXMDSoASAFQAw%3D%3D"

# EXECUTE

In [ ]:
#initiate browser
driver = webdriver.Chrome(options=chrome_options)

# first screenshoot
path_first_screenshoot = make_screenshot(driver, get_gmaps_link(lat_test, long_test, zoom1), f"{path_output}/screenshoot_zoom_1.png")

# Periksa ukuran resolusi gambar
res_x, res_y = get_image_resolution(path_first_screenshoot)

# Import SHP
geo_df_kota = gpd.read_file(path_shp)
geo_df_kota.head()
geo_kelurahan = geo_df_kota[(geo_df_kota["WADMKC"]==kecamatan) & (geo_df_kota["WADMKD"]==kelurahan)]["geometry"]

# grid size
x1_size, y1_size = calculate_grid_size(res_x,res_y,zoom1)
x2_size, y2_size = calculate_grid_size(res_x,res_y,zoom2)
x3_size, y3_size = calculate_grid_size(res_x,res_y,zoom3)

# make grid maps
grid1 = create_grid(geo_kelurahan,x1_size,y1_size)
grid3 = create_grid(geo_kelurahan,x3_size,y3_size)

# centroid of each grid
grid1 = get_centroid(grid1)
grid3 = get_centroid(grid3)

# Initiate Dataframe hasil
df_result = pd.DataFrame()

# looping 1
for i in range(len(grid3)):
    # l = get_gmaps_link(round(grid3['centroid'][i].y,7), round(grid3['centroid'][i].x,7),zoom3)
    # img = make_screenshot(driver, l,f"{path_output}screenshoot/zoom3_{i+1}.png")
    img=f"{path_output}screenshoot/zoom3_{i+1}.png"
    
    result = detect_text(img)
    save_image_with_boxes(img,result['boxes'],f"{path_output}ocr_result/zoom1_{i+1}.png")
    
    result2 = detect_text(img, bin=True, inv=True)
    save_image_with_boxes(img,result2['boxes'],f"{path_output}ocr_result2/zoom1_{i+1}.png")
    
    df_result = pd.concat([df_result, result], ignore_index=True, axis=0)
    df_result = pd.concat([df_result, result2], ignore_index=True, axis=0)

    clean_result = cleaning_text(result)

# looping 3
for i in range(len(grid3)):
    # l = get_gmaps_link(round(grid3['centroid'][i].y,7), round(grid3['centroid'][i].x,7),zoom3)
    # img = make_screenshot(driver, l,f"{path_output}screenshoot/zoom3_{i+1}.png")
    img=f"{path_output}screenshoot/zoom3_{i+1}.png"
    
    result = detect_text(img)
    save_image_with_boxes(img,result['boxes'],f"{path_output}ocr_result/zoom3_{i+1}.png")
    
    result2 = detect_text(img, bin=True, inv=True)
    save_image_with_boxes(img,result2['boxes'],f"{path_output}ocr_result2/zoom3_{i+1}.png")
    
    df_result = pd.concat([df_result, result], ignore_index=True, axis=0)
    df_result = pd.concat([df_result, result2], ignore_index=True, axis=0)

    clean_result = cleaning_text(result)

driver.quit()

In [31]:
df_result.head(20)

,text,scores,boxes,centroid
0,6 Syamsudin. SH,0.910833,"[[1656.0, 0.0], [1779.0, 31.0], [1772.0, 56.0]...",POINT (1716.5611413043478 27.23641304347826)
1,I,0.813640,"[[24.0, 26.0], [46.0, 26.0], [46.0, 46.0], [24...",POINT (35 36)
2,Telusuri Google Maps,0.975613,"[[108.0, 22.0], [248.0, 24.0], [248.0, 46.0], ...",POINT (178 34)
3,Q,0.942697,"[[376.0, 24.0], [394.0, 24.0], [394.0, 44.0], ...",POINT (385 34)
4,JI.Syamsu,0.931889,"[[1836.0, 58.0], [1919.0, 88.0], [1914.0, 110....",POINT (1873.9240506329113 83.9493670886076)
5,Disimpar,0.992039,"[[12.0, 104.0], [58.0, 104.0], [58.0, 120.0], ...",POINT (35 112)
6,Like Earth Coffee,0.979770,"[[1232.0, 112.0], [1360.0, 112.0], [1360.0, 13...",POINT (1296 123)
7,Terbar,0.997732,"[[16.0, 177.0], [50.0, 177.0], [50.0, 191.0], ...",POINT (33 184)
8,Maxan xie oba,0.615168,"[[1332.0, 221.0], [1456.0, 221.0], [1456.0, 25...",POINT (1394 239)
9,Miamay Crunchy,0.865180,"[[1485.0, 235.0], [1614.0, 239.0], [1613.0, 26...",POINT (1549 251)
